In [1]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]

In [66]:
product = "Biologische brocolli"
transactions[transactions["description"] == product]

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
237,2018-01-01,13:03:28,16.0,NaN,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
420,2018-01-01,14:09:57,24.0,55280334.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
683,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
699,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
1057,2018-01-01,19:59:10,58.0,53245671.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
353636,2018-12-29,17:25:54,20625.0,90982137.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
353638,2018-12-29,17:25:54,20625.0,90982137.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
353642,2018-12-29,17:25:54,20625.0,90982137.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
353908,2018-12-30,9:09:23,20639.0,6301858.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0


In [67]:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return converted_date

inventory["date"] = inventory.apply(date_converter, axis = 1)

In [68]:
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

inventory["week"] = inventory.apply(week_of_year, axis = 1)

In [69]:
filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
promotions_filter = promotions[promotions["description"] == product]

def check_discount(row):
    if row["week"] in promotions_filter["week"].values:
        discount_amount = promotions_filter[promotions_filter["week"] == row["week"]]["discount"]
        return discount_amount.values[0]
    else:
        return 0

filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)


<ipython-input-69-bc6caeb2001c>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
def total_inv(row):
    total = 0
    if len(row[product]) > 0:
        for batch in row[product]:
            total += batch[1]
    return total

filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)

<ipython-input-70-24d8929c98e7>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
best_before = []
for element_list in filter_inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = np.unique(best_before)

In [72]:
df_waste = pd.DataFrame({"best before":bb_dates})
df_waste

,best before
0,14
1,17
2,21
3,24
4,28
...,...
99,360
100,364
101,367
102,371


In [73]:
def input_inv(row):
    for element_list in filter_inventory[product]:
        if len(element_list) > 0:
            for batch in element_list:
                if batch[0] == row["best before"]:
                    return batch[1]
df_waste["amount"] = df_waste.apply(input_inv, axis = 1)

In [74]:
df_waste
#product gets thrown away on the day it goes bad, no one can buy it anymore ON THAT DAY
#discard any items that has best before for enxt year
#haven't sold anything that is not in stock
df_waste = df_waste[df_waste["best before"] < 365]
df_waste

,best before,amount
0,14,26
1,17,26
2,21,26
3,24,26
4,28,26
...,...,...
96,350,26
97,353,26
98,357,26
99,360,26


In [75]:
transactions["day"] = pd.to_datetime(transactions["day"])
filter_transactions = transactions[transactions["description"] == product]

#filter_transactions["week"] = filter_transactions.apply(week_of_year, axis=1)
first_restock = df_date[df_date["day"] == 3]["date"].values[0]
#first_restock
filter_transactions[filter_transactions["day"] < first_restock]
#PROBLEM: IF A PRODUCT HASN'T BEEN BOUGHT IN ONE DAY IT HAS A ZERO, CAUSES AN ERROR FEW CELLS 

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
237,2018-01-01,13:03:28,16.0,NaN,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
420,2018-01-01,14:09:57,24.0,55280334.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
683,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
699,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
1057,2018-01-01,19:59:10,58.0,53245671.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
29070,2018-01-02,12:35:20,1713.0,17960423.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0
56620,2018-01-03,18:38:07,3371.0,33780213.0,vegetable,v_20,Biologische brocolli,400g,1.79,1.79,1.0,0.0


In [76]:
filter_transactions["date"].iloc[0].day

KeyError: 'date'

In [77]:
purchases_per_day = filter_transactions.groupby("day").count()["product_id"].cumsum()
purchases_per_day_plotting = filter_transactions.groupby("day").count()["product_id"]
purchases_per_day_plotting.index = purchases_per_day_plotting.index.dayofyear/7
purchases_per_day_plotting

day
0.142857     5
0.285714     1
0.428571     1
0.571429     4
0.714286     4
            ..
51.428571    5
51.571429    1
51.714286    1
51.857143    5
52.000000    2
Name: product_id, Length: 333, dtype: int64

In [78]:
purchases_per_day.index[0].year
weeknr = []
for ind in purchases_per_day.index:
    week = datetime.date(ind.year, ind.month, ind.day).isocalendar()[1]
    weeknr.append(week)


In [81]:
df_waste.drop(index=df_waste.tail(9).index, inplace=True)
df_waste

C:\Users\zolyo\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,best before,amount
0,14,26
1,17,26
2,21,26
3,24,26
4,28,26
...,...,...
83,304,26
84,308,26
85,311,26
86,315,26


In [82]:
df_waste["purchases"] = [purchases_per_day[i-1] for i in df_waste["best before"]]

<ipython-input-82-aaa3093af6f6>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [83]:
df_waste["prev purchases"] = df_waste["purchases"].diff()
df_waste

<ipython-input-83-202d6bfb1fb9>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,best before,amount,purchases,prev purchases
0,14,26,52,NaN
1,17,26,62,10.0
2,21,26,81,19.0
3,24,26,87,6.0
4,28,26,105,18.0
...,...,...,...,...
83,304,26,1196,12.0
84,308,26,1210,14.0
85,311,26,1224,14.0
86,315,26,1243,19.0


In [84]:
length = df_waste.shape[0]
lst_waste = []
df_waste["remaining"] = df_waste["amount"] - df_waste["prev purchases"]
df_waste["remaining"].iloc[0] = df_waste["amount"].iloc[0] - df_waste["purchases"].iloc[0]

for i in range(length):
    current_row = df_waste.iloc[i]
    if i == 0:
        lst_waste.append(current_row["remaining"])
        continue     
    else:
        prev_row = df_waste.iloc[i-1]
        if lst_waste[-1] < 0:
            waste = lst_waste[-1] + current_row["remaining"]
            lst_waste.append(lst_waste[-1] + current_row["remaining"])
        else:
            lst_waste.append(current_row["remaining"])
    
df_waste["waste"] = lst_waste
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["best before"])
    return converted_date

df_waste["date"] = df_waste.apply(date_converter, axis = 1)
df_waste["week"] = df_waste.apply(week_of_year, axis = 1)
df_waste[df_waste["waste"] < 0]["waste"] = 0

<ipython-input-84-15cae0269777>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\zolyo\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-84-15cae0269777>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-84-15cae0269777>:19: SettingWithCopyWarning:


A value is trying to b

In [85]:
df_waste

,best before,amount,purchases,prev purchases,remaining,waste,date,week
0,14,26,52,NaN,-26.0,-26.0,2018-01-15,3
1,17,26,62,10.0,16.0,-10.0,2018-01-18,3
2,21,26,81,19.0,7.0,-3.0,2018-01-22,4
3,24,26,87,6.0,20.0,17.0,2018-01-25,4
4,28,26,105,18.0,8.0,8.0,2018-01-29,5
...,...,...,...,...,...,...,...,...
83,304,26,1196,12.0,14.0,14.0,2018-11-01,44
84,308,26,1210,14.0,12.0,12.0,2018-11-05,45
85,311,26,1224,14.0,12.0,12.0,2018-11-08,45
86,315,26,1243,19.0,7.0,7.0,2018-11-12,46


In [86]:
df_waste["waste nn"] = [i if i > 0 else 0 for i in df_waste["waste"]]
waste_group = df_waste.groupby("week").sum()


<ipython-input-86-aae312e1e342>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [87]:
waste_group["week"] = waste_group.index

In [89]:
import plotly.express as px
fig = px.line(waste_group, x = "week", y = "waste nn", title="Absolute waste of Bio broccoli per week")
fig.update_yaxes(title="Waste amount")
fig.show()

In [112]:
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [71]:
transactions["day"] = pd.to_datetime(transactions["day"])
filter_transactions = transactions[transactions["description"] == product]

In [86]:
daydate = df_date[df_date["day"] == 2]["date"].values[0]

filter_transactions[filter_transactions["day"] <= daydate]


,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
89,2018-01-01,10:40:13,6.0,54211938.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
467,2018-01-01,14:21:35,26.0,51433968.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
573,2018-01-01,15:36:46,33.0,30794082.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
700,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
808,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
821,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
824,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
879,2018-01-01,18:46:48,49.0,15382238.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29093,2018-01-02,12:52:06,1714.0,28719528.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29134,2018-01-02,13:44:14,1718.0,39309224.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0


In [111]:
purchases_per_day = filter_transactions.groupby("day").count()["product_id"].cumsum()
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [52]:
daydate

numpy.datetime64('2018-01-15T00:00:00.000000000')

In [13]:
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})
df_date

,day,date
0,0,2018-01-01
1,1,2018-01-02
2,2,2018-01-03
3,3,2018-01-04
4,4,2018-01-05
...,...,...
360,360,2018-12-27
361,361,2018-12-28
362,362,2018-12-29
363,363,2018-12-30
